In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium 
import requests
import json
from geopy.distance import geodesic

hdb_resale_prices = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

<h3>Data Preparation</h3>

In [ ]:
hdb_resale_prices

<h5><b>Checking for Missing Values</b></h5>

There are no missing values in the dataset.

In [ ]:
hdb_resale_prices.isnull().sum()

In [ ]:
hdb_resale_prices.info()

In [ ]:
hdb_resale_prices.describe()

<h1>Exploratory Data Analysis</h1>

### Distribution of Resale Prices
<h5><b>Use mean or median resale price?</b></h5>

As seen in the plotted histogram for the resale prices from 2017 to 2023, the resale price data is right skewed. Since the price data does not follow a symmetric distribution, we will use the <i><b>median price</b></i> instead of the mean price to represent the average resale price of the flats.

The right-skewed KDE plot also suggests that the mean prices are greater than the median.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x=hdb_resale_prices['resale_price'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.histplot(data=hdb_resale_prices, x="resale_price", ax=ax, kde=True)

<h3>Flat Type Distribution</h3>

Based on the plotted bar chart, the majority of the resale flats are 4-room flats and multi-generation flats account for the least number of resale flats.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.countplot(x=hdb_resale_prices['flat_type'], data=hdb_resale_prices)
abs_values = hdb_resale_prices['flat_type'].value_counts(ascending=False).values
ax.bar_label(container=ax.containers[0], labels=abs_values)
plt.show()

In [ ]:
# Since young couples are likely to purchase 3,4,5 room and executive flats we decide to drop the 1 room, 2room and multi-gen flats
hdb_resale_prices = hdb_resale_prices[hdb_resale_prices['flat_type'] != '1 ROOM']
hdb_resale_prices = hdb_resale_prices[hdb_resale_prices['flat_type'] != '2 ROOM']
hdb_resale_prices = hdb_resale_prices[hdb_resale_prices['flat_type'] != 'MULTI-GENERATION']

hdb_resale_prices.reset_index(inplace=True)

In [ ]:
hdb_resale_prices.head()

In [ ]:
df_coordinates = pd.read_csv('help_la_lat_long.csv')
df_coordinates = df_coordinates.rename(columns={'Latitude':0, 'Longitude':1})
df_coordinates.rename(columns={"Unnamed: 0": 'index'}, inplace=True)
df_coordinates.head()


In [ ]:
df_combined = hdb_resale_prices.merge(df_coordinates, how='left', on=["index", "month", "flat_type", "storey_range", "floor_area_sqm", "flat_model", "resale_price", "town"])
df_combined = df_combined.drop(columns="index")
df_combined = df_combined.rename(columns={0:'Latitude', 1:'Longitude'})

In [ ]:
df_combined.head()

In [ ]:
# plot distribution of prices against flat type 
fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x=hdb_resale_prices['flat_type'], y=hdb_resale_prices['resale_price'])

### Investigating the distribution of flat_model

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.countplot(y='flat_model', data=df_combined)
abs_values = df_combined['flat_model'].value_counts(ascending=False).values
ax.bar_label(container=ax.containers[0], labels=abs_values)
plt.show()

# Feature Engineering

Based on the studies and analysis performed by other researchers, the following features that have been concluded to be <b>important</b> in determining the prices of HDB resale flats are as follows: 
- Time Taken to Travel to Central Business District (CBD), Raffles Place
- Number of Years Left on Lease
- Maturity of Estate
<br>

We are also considering <b>other factors</b> that have not been explored by other reseearchers. The factors that we have identified are as follows: 
- Singapore's Inflation Rate
- Flat Model
- Floor Level
- Prices of BTO
- Number of BTO Flats Launched that Year
- Demand Ratio in Resale Flat


Before adding these features into our dataset, we will first perform some data cleaning and feature engineering on the existing features in our dataset.

- convert the flat condition with quantitative measures
- combine the resale prices and the sqm to find the $/sqm (still need to keep sqm to quantify the size!!) 
- remaining lease should be converted to a ratio 
- combine the block and street name to obtain the lat,long - from there we can obtain the distance from amentities, distance from public transport, distance from central
- using the town --- can also do some quantitative measures to quantify how mature the estate is
- average price of the houses of the same category in the area from here we derive the estimated COV
- demand-scarcity ratio --- not sure how to quantify this
- total sale price of the hdb
- there should be a way to convert the storey range to scoring value

### Incorporating Inflation Rate into Resale Prices

In [ ]:
valid_rows = [10,11,14]
cpi_df = pd.read_csv("cpi-2012-to-2022.csv", skiprows=lambda x: x not in valid_rows, index_col=0)

cpi_df = cpi_df.transpose()
cpi_df

In [ ]:
# convert datetime index to year
cpi_df.index = pd.DatetimeIndex(cpi_df.index).year

In [ ]:
# add new column to hdb_resale_prices dataframe to store consumer price index for housing and utilities of corresponding year
latest_year = 2022
df_combined['cpi_multiplier'] = df_combined["month"].apply(lambda x: cpi_df["Housing & Utilities"].loc[min(int(x.split("-")[0]), latest_year)] / 100)

df_combined.head()

In [ ]:
# add new column to hdb_resale_prices dataframe to store adjusted resale price
df_combined['cpi_adjusted_price'] = df_combined["resale_price"] * df_combined["cpi_multiplier"] / df_combined["floor_area_sqm"]

### Calculating Price Per Square Meter

In [ ]:
# add new column to store adjusted resale price 
df_combined['cpi_adjusted_price_per_sqm'] = df_combined["cpi_adjusted_price"] / df_combined["floor_area_sqm"]
df_combined.head()

### No. of BTO per year


In [ ]:
# Create the dictionary
dictionary ={2017:14464, 2018 : 17556, 2019 : 12114, 2020 : 7314, 2021: 13756,2022:20064,2023: 20000}
 
# Add a new column named 'Price'
df_combined['no.of bto'] = df_combined['year'].map(dictionary)
 
# Print the DataFrame"
print(df_combined)

### Demand for resale


In [ ]:
resale_application = {2017:20894,2018:22005,2019:24772,2020:26436,2021:26436,2022:30370}

In [ ]:
df_combined['resale application'] = df_combined['year'].map(resale_application)

In [ ]:
df_combined

In [ ]:
(df_combined['year']==2017).sum()

In [ ]:
(df_combined['year']==2018).sum()

In [ ]:
(df_combined['year']==2019).sum()

In [ ]:
(df_combined['year']==2020).sum()

In [ ]:
(df_combined['year']==2021).sum()

In [ ]:
(df_combined['year']==2022).sum()

In [ ]:
df_combined.loc[df_combined['year'] == 2017]['resale application']/(df_combined['year']==2017).sum()

In [ ]:
df_combined.loc[df_combined['year'] == 2019]['resale application']/(df_combined['year']==2019).sum()

In [ ]:
df_combined.loc[df_combined['year'] == 2020]['resale application']/(df_combined['year']==2020).sum()

In [ ]:
df_combined.loc[df_combined['year'] == 2021]['resale application']/(df_combined['year']==2021).sum()

In [ ]:
df_combined.loc[df_combined['year'] == 2022]['resale application']/(df_combined['year']==2022).sum()

In [ ]:
demand_ratio = {2017:1.02,2018:1.02,2019:1.12,2020:1.13,2021:0.91,2022:1.14}

In [ ]:
df_combined['demand ratio'] = df_combined['year'].map(demand_ratio)

In [ ]:
hdb_resale_prices.to_excel(r'resaleWithDemandAndBtoSupply.xlsx', index=False)

<h3>Remaining Lease Months</h3>

In [ ]:
for i in range(len(df_combined)):
    if "months" in df_combined.loc[i,"remaining_lease"]:
        
        df_combined.loc[i,"remaining_lease_months"]= int(str(df_combined.loc[i,"remaining_lease"]).split(" ")[0])*12 +int(str(df_combined.loc[i,"remaining_lease"]).split(" ")[2])
    else:
        df_combined.loc[i,"remaining_lease_months"]= int(str(df_combined.loc[i,"remaining_lease"]).split("years")[0])*12

In [ ]:
df_combined= df_combined.drop(columns=["lease_commence_date","remaining_lease"])

<h3>Storey Range</h3>

In [ ]:
for i in range(len(df_combined)):
    total = int(str(df_combined.loc[i,'storey_range']).split('TO')[1])+ int(str(df_combined.loc[i,'storey_range']).split('TO')[0])
    df_combined.loc[i,'storey_range'] = total / 2

<h3>Geo Coding</h3>

In [ ]:
df_combined['address'] = df_combined['block'] + " " + df_combined['street_name']

In [ ]:
df_combined= df_combined.drop(columns=["block","street_name"])

In [ ]:
addressList = list(df_combined['address'])

<h3>Distance from nearest mrt</h3>

In [ ]:
list_of_mrt = ["Jurong East MRT",
              "Bukit Batok MRT",
              "Bukit Gombak MRT",
              "Choa Chu Kang MRT",
              "Yew Tee MRT",
              "Kranji MRT",
              "Marsiling MRT",
              "Woodlands MRT",
              "Admiralty MRT",
              "Sembawang MRT",
              "Canberra MRT",
              "Admiralty MRT",
              "Sembawang MRT",
              "Canberra MRT",
              "Yishun MRT",
              "Khatib MRT",
              "Yio Chu Kang MRT",
              "Ang Mo Kio MRT",
              "Bishan MRT",
              "Braddell MRT",
              "Toa Payoh MRT",
              "Novena MRT",
              "Newton MRT",
              "Orchard MRT",
              "Somerset MRT",
              "Dhoby Ghaut MRT",
              "City Hall MRT",
              "Raffles Place MRT",
              "Marina Bay MRT",
              "Marina South Pier MRT",
               "Tuas Link MRT",
               "Tuas West Road MRT",
               "Tuas Crescent MRT",
               "Gul Circle MRT",
               "Joo Koon MRT",
               "Pioneer MRT",
               "Boon Lay MRT",
               "Lakeside MRT",
               "Chinese Garden MRT",
               "Clementi MRT",
               "Dover MRT",
               "Buona Vista MRT",
               "Commonwealth MRT",
               "Queenstown MRT",
               "Redhill MRT",
               "Tiong Bahru MRT",
               "Outram Park MRT",
               "Tanjong Pagar MRT",
               "Raffles Place MRT",
               "Bugis MRT",
               "Lavender MRT",
               "Kallang MRT",
               "Aljunied MRT",
               "Paya Lebar MRT",
               "Eunos MRT",
               "Kembangan MRT",
               "Bedok MRT",
               "Tanah Merah MRT",
               "Simei MRT",
               "Tampines MRT",
               "Pasir Ris MRT",
               "Expo MRT",
               "Changi Airport MRT",
               "HarbourFront MRT",
               "Chinatown MRT",
               "Clarke Quay MRT",
               "Little India MRT",
               "Farrer Park MRT",
               "Boon Keng MRT",
               "Potong Pasir MRT",
               "Woodleigh MRT",
               "Serangoon MRT",
               "Kovan MRT",
               "Hougang MRT",
               "Buangkok MRT",
               "Sengkang MRT",
               "Punggol MRT",
               "Punggol Coast MRT",
               "Haw Par Villa MRT",
               "Holland Village MRT",
               "Kent Ridge MRT",
               "one-north MRT",
               "Farrer Road MRT",
               "Botanic Gardens MRT",
               "Caldecott MRT",
               "Marymount MRT",
               "Bishan MRT",
               "Lorong Chuan MRT",
               "Bartley MRT",
               "Tai Seng MRT",
               "MacPherson MRT",
               "Dakota MRT",
               "Mountbatten MRT",
               "Stadium MRT",
               "Nicoll Highway MRT",
               "Promenade MRT",
               "Esplanade MRT",
               "Bras Basah MRT",
               "Pasir Panjang MRT",
               "Labrador Park MRT",
               "Telok Blangah MRT",
               "Bayfront MRT",
               "Bukit Panjang MRT",
               "Cashew MRT",
               "Hillview MRT",
               "Beauty World MRT",
               "King Albert Park MRT",
               "Sixth Avenue MRT",
               "Tan Kah Kee MRT",
               "Stevens MRT",
               "Rochor MRT",
               "Downtown MRT",
               "Telok Ayer MRT",
               "Fort Canning MRT",
               "Bencoolen MRT",
               "Jalan Besar MRT",
               "Bendemeer MRT",
               "Geylang Bahru MRT",
               "Mattar MRT",
               "Ubi MRT",
               "Kaki Bukit MRT",
               "Bedok North MRT",
               "Bedok Reservoir MRT",
               "Tampines West MRT",
               "Tampines East MRT",
               "Upper Changi MRT",
               "Woodlands North MRT",
               "Woodlands South MRT",
               "Springleaf MRT",
               "Lentor MRT",
               "Mayflower MRT",
               "Bright Hill MRT",
               "Upper Thomson MRT",
               "Napier MRT",
               "Orchard Boulevard MRT",
               "Great World MRT",
               "Havelock MRT",
               "Maxwell MRT",
               "Shenton Way MRT",
               "Gardens by the Bay MRT"
               
              ]


In [ ]:

mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)
    
    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

In [ ]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

In [ ]:
list_of_mrt_coordinates = []
list_of_lat = df_coordinates[0]
list_of_long = df_coordinates[1]
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']


list_of_coordinates = []
for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

In [ ]:
# nearest mrt 
# pip install geopy
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [ ]:
df_combined["Distance to nearest MRT"] = min_dist_mrt

<h3>Distance to CBD</h3>

In [ ]:
# Central Business District (CBD) The CBD is Singapore's business and financial district, and home to leading international businesses and financial institutions. It spans from Raffles Place along Shenton Way / Robinson Road / Cecil Street to the Tanjong Pagar and Anson subzones.
list_of_cbd=[
    "Shenton Way",
    "Raffles Place",
    "Robinson Road",
    "Cecil Street",
    "Tanjong Pagar"
]

In [ ]:
cbd_lat = []
cbd_long = []

for i in range(0, len(list_of_cbd)):
    query_address = list_of_cbd[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_cbd=json.loads(resp.content)
    
    if data_cbd['found'] != 0:
        cbd_lat.append(data_cbd["results"][0]["LATITUDE"])
        cbd_long.append(data_cbd["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_cbd['results'][0]['LATITUDE'] +" Long: "+data_cbd['results'][0]['LONGITUDE'])

    else:
        cbd_lat.append('NotFound')
        cbd_long.append('NotFound')
        print ("No Results")


In [ ]:
cbd_location = pd.DataFrame({
    'latitude': cbd_lat,
    'longitude': cbd_long
})

In [ ]:
# Lists of all the coordinates we will need to iterate through
list_of_cbd_coordinates = []
list_of_lat = df_coordinates[0]
list_of_long = df_coordinates[1]
cbd_lat = cbd_location['latitude']
cbd_long = cbd_location['longitude']
# Zipping the respective Lat and Long lists together as a list of tuples
list_of_coordinates = []
for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(cbd_lat, cbd_long):
    list_of_cbd_coordinates.append((lat, long))

In [ ]:


list_of_dist_cbd = []
min_dist_cbd = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_cbd_coordinates)):
        list_of_dist_cbd.append(geodesic(origin,list_of_cbd_coordinates[destination]).meters)
    shortest = (min(list_of_dist_cbd))
    min_dist_cbd.append(shortest)
    list_of_cbd.clear()

In [ ]:
df_combined["Distance to CBD"] = min_dist_cbd

<h3>Distance to nearest Mall</h3>

In [ ]:
list_of_malls = [

"100 AM",
"313@Somerset",
"Aperia",
"Balestier Hill Shopping Centre",
"Bugis Cube",
"Bugis Junction",
"Bugis+",
"Capitol Piazza",
"Cathay Cineleisure Orchard",
"Clarke Quay Central",
"The Centrepoint",
"City Square Mall",
"City Gate Mall",
"CityLink Mall",
"Duo",
"Far East Plaza",
"Funan",
"Great World City",
"HDB Hub",
"Holland Village Shopping Mall",
"ION Orchard",
"Junction 8",
"Knightsbridge",
"Liat Towers",
"Lucky Plaza",
"Marina Bay Sands",
"The Shoppes at Marina Bay Sands",
"Marina Bay Link Mall",
"Marina Square",
"Millenia Walk",
"Mustafa Shopping Centre",
"Ngee Ann City",
"Orchard Central",
"Orchard Gateway",
"Orchard Plaza",
"Midpoint Orchard",
"Palais Renaissance",
"People's Park Centre",
"People's Park Complex",
"Plaza Singapura",
"PoMo",
"Raffles City",
"Scotts Square",
"Shaw House and Centre",
"Sim Lim Square",
"Singapore Shopping Centre",
"The South Beach",
"Square 2",
"Sunshine Plaza",
"Suntec City",
"Tanglin Mall",
"Tanjong Pagar Centre",
"Tekka Centre",
"The Adelphi",
"The Paragon",
"Tiong Bahru Plaza",
"The Poiz",
"Thomson Plaza",
"United Square",
"Thomson V",
"Velocity@Novena Square",
"Wheelock Place",
"Wisma Atria",
"Zhongshan Mall",
"Bedok Mall",
"Century Square",
"Our Tampines Hub",
"Changi City Point",
"Downtown East",
"Djitsun Mall Bedok",
"Eastpoint Mall",
"Jewel Changi Airport",
"KINEX",
"Katong Shopping Centre",
"Katong Square",
"Kallang Wave Mall",
"Leisure Park Kallang",
"i12 Katong",
"Parkway Parade",
"Paya Lebar Square",
"Paya Lebar Quarter",
"Roxy Square",
"Singpost Centre",
"Tampines 1",
"Tampines Mall",
"White Sands",
"City Plaza",
"Elias Mall",
"Loyang Point",
"888 Plaza",
"Admiralty Place",
"AMK Hub",
"Canberra Plaza",
"Causeway Point",
"Woodlands Civic Centre",
"Broadway Plaza",
"Djitsun Mall",
"Jubilee Square",
"Junction 8",
"Junction Nine",
"Marsiling Mall",
"Northpoint City",
"Sembawang Shopping Centre",
"Sun Plaza",
"Vista Point",
"Wisteria Mall",
"Woodlands Mart",
"Woodlands North Plaza",
"Waterway Point",
"Compass One",
"Hougang Mall",
"Heartland Mall",
"NEX",
"Buangkok Square",
"Greenwich V",
"Hougang",
"Hougang Green Shopping Mall",
"Hougang Rivercourt",
"myVillage At Serangoon Garden",
"Northshore Plaza",
"Oasis Terraces",
"Punggol Plaza",
"Rivervale Mall",
"Rivervale Plaza",
"The Seletar Mall",
"Upper Serangoon Shopping Centre",
"Beauty World Centre",
"Beauty World Plaza",
"Bukit Panjang Plaza",
"Bukit Timah Plaza",
"Fajar Shopping Centre",
"Greenridge Shopping Centre",
"Hillion Mall",
"HillV2",
"Junction 10",
"Keat Hong Shopping Centre",
"Limbang Shopping Centre",
"Lot One",
"Rail Mall",
"Sunshine Place",
"Teck Whye Shopping Centre",
"West Mall",
"Yew Tee Point",
"Yew Tee Square",
"VivoCity",
"HarbourFront Centre",
"Alexandra Retail Centre",
"321 Clementi",
"The Clementi Mall",
"IMM",
"JCube",
"Jem",
"Westgate",
"Jurong Point",
"Pioneer Mall",
"The Star Vista",
"Alexandra Central",
"Anchorpoint",
"OD Mall",
"Boon Lay Shopping Centre",
"Grantral Mall",
"Fairprice Hub",
"Gek Poh Shopping Centre",
"Rochester Mall",
"Taman Jurong Shopping Centre",
"West Coast Plaza",
"Queensway Shopping Centre"]

In [ ]:
mall_lat = []
mall_long = []

for i in range(0, len(list_of_malls)):
    query_address = list_of_malls[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mall=json.loads(resp.content)
    
    if data_mall['found'] != 0:
        mall_lat.append(data_mall["results"][0]["LATITUDE"])
        mall_long.append(data_mall["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mall['results'][0]['LATITUDE'] +" Long: "+data_mall['results'][0]['LONGITUDE'])

    else:
        mall_lat.append('NotFound')
        mall_long.append('NotFound')
        print ("No Results ," + query_address)
# no clarke quay central, no OD mall, no city gate mall, holland village shopping mall, mustafa shopping centre, PoMo, ShawHouse and Centre

In [ ]:
mall_location = pd.DataFrame({
    'mall': list_of_malls,
    'latitude': mall_lat,
    'longitude': mall_long
})

In [ ]:
# Lists of all the coordinates we will need to iterate through
list_of_mall_coordinates = []
list_of_lat = df_coordinates[0]
list_of_long = df_coordinates[1]
mall_lat = mall_location['latitude']
mall_long = mall_location['longitude']
# Zipping the respective Lat and Long lists together as a list of tuples
list_of_coordinates = []
for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(cbd_lat, cbd_long):
    list_of_mall_coordinates.append((lat, long))

In [ ]:
list_of_dist_mall = []
min_dist_mall = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mall_coordinates)):
        list_of_dist_mall.append(geodesic(origin,list_of_mall_coordinates[destination]).meters)
    shortest = (min(list_of_dist_cbd))
    min_dist_mall.append(shortest)
    list_of_dist_mall.clear()

In [ ]:
df_combined["Distance to nearest mall"] = min_dist_mall

<h3>IsMature Estate</h3>

In [ ]:
list_mature_estates = ["ang mo kio", 
                        "bishan", 
                        "clementi",
                        "bukit merah",
                        "bedok",
                        "bukit timah",
                        "geylang",
                        "central",
                        "marine parade",
                        "queenstown",
                        "pasir ris",
                        "kallang" , "whampoa", "marine parade" , "serangoon", "tampines", "toa payoh"]

for i in range(len(df_combined)):
    if df_combined.loc[i,'town'].lower() in list_mature_estates:
        df_combined.loc[i,'isMatureEstate'] = 1
    else:
        df_combined.loc[i,'isMatureEstate'] = 0

In [ ]:
df_combined.to_csv('final_hdb_resale_prices.csv')

### Correlation

In [ ]:
plt.figure(figsize=(16, 6))
corr_df = df_combined.corr()
hm = sns.heatmap(corr_df, annot=True)

### Investigate if lease is an important factor

In [ ]:
import csv
hdb_resale_listings = []

with open('final_hdb_resale_prices.csv', newline='', encoding="utf8") as f:
    reader = csv.reader(f)
    for row in reader:
        hdb_resale_listings.append(row)

    # Remove header
    hdb_resale_listings_headers = hdb_resale_listings[0]
    hdb_resale_listings = hdb_resale_listings[1:]
    
print(hdb_resale_listings[0]) # Print first row of data

In [ ]:
average_resale_for_lease_year_remaining = {}
for data in hdb_resale_listings:
    if round(float(data[16])/12) not in average_resale_for_lease_year_remaining:
        average_resale_for_lease_year_remaining[round(float(data[16])/12)] = []
        average_resale_for_lease_year_remaining[round(float(data[16])/12)].append(float(data[7]))
    else:
        average_resale_for_lease_year_remaining[round(float(data[16])/12)].append(float(data[7]))
    

In [ ]:
for item in average_resale_for_lease_year_remaining:
    avg = round(sum(average_resale_for_lease_year_remaining[item])/len(average_resale_for_lease_year_remaining[item]))
    average_resale_for_lease_year_remaining[item] = avg

In [ ]:
lease = list(average_resale_for_lease_year_remaining.keys())
average_resale_value = list(average_resale_for_lease_year_remaining.values())

## Write your code below
fig,ax = plt.subplots(figsize=(18,12))
ax.plot(lease,average_resale_value,marker='d')
ax.set_ylabel('Resale Value')
ax.set_xlabel('Lease Remaining')
ax.set_title('Average Resale Value for Lease Years Remaining')
plt.show()


### Investigate how floor area correlate with price

In [ ]:
floor_area_sqm = []
resale_price = []
for data in hdb_resale_listings:
    floor_area_sqm.append(float(data[5]))
    resale_price.append(float(data[7]))
fig,ax = plt.subplots(figsize=(18,12))
ax.scatter(floor_area_sqm,resale_price)
ax.set_xlabel('Floor Area (in square metres)')
ax.set_ylabel('Sale Price (in million SGD)')
ax.set_title('Relationship between Floor Area and Resale Price')
plt.show()


### Investigating resale counts by town

In [ ]:
resale_counts_by_town = {}  # Populate this dictionary with counts

import operator
## Write your code below

for data in hdb_resale_listings:
    hdb_town = data[2]
    
    if hdb_town in resale_counts_by_town:
        
        resale_counts_by_town[hdb_town] += 1 
    
    else:
        
        resale_counts_by_town[hdb_town] = 1

    
        
sorted_resale_counts_by_town = dict(sorted(resale_counts_by_town.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
resale_counts_sorted_desc = sorted_resale_counts_by_town
neighbourhood_listings = list(resale_counts_sorted_desc.keys())
num_listings = list(resale_counts_sorted_desc.values())
# Import matplotlib library here

import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize=(18,12))
ax.bar(neighbourhood_listings,num_listings)
ax.set_title('Counts of Resale Flats from Jan 2017 to June 2020')
ax.set_xticklabels(neighbourhood_listings,rotation = 90)
ax.set_ylabel('Number of Resale Flats')
ax.set_xlabel('Town')
plt.show()

# Folium Clusters

<h3>3 Room</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == '3 ROOM':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)

In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == '3 ROOM':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)


<h3>4 Room</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == '4 ROOM':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)



In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == '4 ROOM':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)


<h3>5 Room</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == '5 ROOM':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)

In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == '5 ROOM':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)

<h3>Executive</h3>

In [ ]:
import folium
from folium.plugins import HeatMap
lat_long = []
## Write your code below
for data in hdb_resale_listings:
    if data[3] == 'EXECUTIVE':
        lat_long.append([data[8],data[9]])
map_folium = folium.Map(location=[1.357,103.826], height=350, width=800,zoom_start=11.4)

HeatMap(lat_long, radius=8, gradient={0.2:'blue', 0.4:'purple', 0.6:'orange', 1.0:'red'}).add_to(map_folium) 

display(map_folium)

In [ ]:
from folium.plugins import MarkerCluster   
## Write your code below (Note: the Folium map output might take a while to load)
map_folium = folium.Map(location=[1.357,103.826], height=550, width=900, zoom_start=11.5)

marker_cluster = MarkerCluster().add_to(map_folium)
for data in hdb_resale_listings:
    if data[3] == 'EXECUTIVE':
        lat_long_listing = [data[8],data[9]]
        pop_display_price = '$' + str(data[6])
        tooltipmsg = 'Click to view price'

        folium.Marker(lat_long_listing,popup=pop_display_price,tooltip = tooltipmsg).add_to(marker_cluster)

display(map_folium)

<h1>Preliminary Models</h1>


In [ ]:
#split training and test data